In [4]:
import pandas as pd
import numpy as np
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import FastText
from epochlogger import EpochLogger
from sentencegenerator import SentenceGenerator

In [6]:
data = pd.read_csv('../data/mfp-diaries-shuffled.tsv',sep='\t',header=None,chunksize=40000)

In [3]:
fasttext = FastText.load('../fasttextmodels/model_foods_as_sentences_train_only_fasttext.model')

In [8]:
chunk


,0,1,2,3
0,2685,2015-01-03,"[{""meal"": ""Breakfast"", ""dishes"": [{""nutritions...","{""total"": [{""name"": ""Calories"", ""value"": 1818}..."
1,5809,2015-02-01,"[{""meal"": ""8:01am - 12:00pm"", ""dishes"": [{""nut...","{""total"": [{""name"": ""Calories"", ""value"": 1331}..."
2,4181,2015-03-21,"[{""meal"": ""Breakfast"", ""dishes"": [{""nutritions...","{""total"": [{""name"": ""Calories"", ""value"": 1753}..."
3,1342,2014-10-13,"[{""meal"": ""Breakfast"", ""dishes"": [{""nutritions...","{""total"": [{""name"": ""Calories"", ""value"": 610},..."
4,8473,2014-11-21,"[{""meal"": ""7a - 10"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 1535}..."
5,9694,2015-01-20,"[{""meal"": ""Breakfast"", ""dishes"": [{""nutritions...","{""total"": [{""name"": ""Calories"", ""value"": 1730}..."
6,8003,2015-01-09,"[{""meal"": ""Breakfast"", ""dishes"": [{""nutritions...","{""total"": [{""name"": ""Calories"", ""value"": 1465}..."
7,4152,2014-12-16,"[{""meal"": ""Breakfast"", ""dishes"": [{""nutritions...","{""total"": [{""name"": ""Calories"", ""value"": 0}, {..."
8,2101,2014-10-05,"[{""meal"": ""Lunch"", ""dishes"": [{""nutritions"": [...","{""total"": [{""name"": ""Calories"", ""value"": 1004}..."
9,2648,2014-12-03,"[{""meal"": ""Lunch"", ""dishes"": [{""nutritions"": [...","{""total"": [{""name"": ""Calories"", ""value"": 1099}..."


In [10]:
i = 0
# with open("../data/food_vectors_context_only.tsv", "w") as sentences:
#     for chunk in data:
chunk = data.get_chunk(10)
for row in chunk.iterrows():
    vectors = []
    ex_diary = json.loads(row[1][2])
    for meal in ex_diary:
        for dish in meal['dishes']:
            
            if dish['name'][0:20] == 'Quick Added Calories':
                provider = 'Quick Added Calories'
                item = 'Quick Added Calories'
            else:
                name, quant = dish['name'].split(',',maxsplit=1)
                try:
                    provider, item = name.split(' - ')[0:2]
                except ValueError:
                    if len(name.split(' - ')) ==1:
                        provider = name
                        item = ""
            sentence = ' '.join(provider.split()) + ' ' + ' '.join(item.split())
            print(sentence)
            for word in sentence:
                try:
                    vectors.append(fasttext.wv[word])
                except KeyError:
                    print(word + "not in vocabulary, omitting")
    avg_vector = np.mean(vectors)
    sentences.write(sentence+'\n')
    i += 1
    print(i,end='\r')

Herbalife Herbal Tea Concentrate/raspberry
Egg Large
Shurfine Country Gravy Mix
Biscuits Plain or buttermilk
Original Pork Sausage Patties Original Pork Sausage Patties
Fruit Banana (7" to 7-7/8" Inch)
Maruchan Instant Lunch Chicken Flavor
Nestle Nesquik Chocolate Lowfat Milk
Snackpack Chocolate Pudding
Dannon Light & Fit Greek Yogurt 5.3oz Cherry Greek Yogurt
Blue Bonnet**** Margarine Spread
Nature's Own Wheat Bread
Coffee Brewed from grounds
Almond Breeze (Final Revision) Unsweetened Vanilla Almond Milk
Bodyfortress Whey Protein-chocolate
Kellogg's Eggo Pancake Bites Chocolate Chip
Ham Baked
Flatout Hungry Girl Foldit Traditional White
Pineapple Raw
Subway Roast Beef Chopped Salad
Lay's Baked
Subway Fountain Cherry Coke
Walmart Great Value Buttermilk Waffles
Walmart Great Value Light Whipped Topping
Subway Large Fountain Diet Coke
Rold Gold® Tiny Twist Pretzels
Great Value Turkey Pastrami
Thomas' Mini Bagels (Plain)
Giant" "Sweet Cream Butter" (Unsalted) Stick
Trader Joe's Organic Vi